for MongoBD
1. install homebrew (https://brew.sh)
2. install MangoBD https://www.youtube.com/watch?v=s1WQ0eEpqqg
3. download, unzip, place the folder in the working dir https://github.com/datacoon/russiannames/tree/master/data/bson
4. in the terminal write: brew services start mongodb-community@6.0
5. in the terminal write: mongorestore -d names
6. after running the whole code, enter brew services stop mongodb-community@6.0 (in the terminal)

# 1. Подгружаем текстовый файл

In [1]:
with open('андрей_белый_москва.txt') as text:
    book = text.read()

# 2. Создаем пока что пустой список из имен

In [2]:
extracted_names = []

# 3. Работаем со Spacy 

In [3]:
# Здесь мы устанавливаем нлп модель для русского языка, тк она по умолчанию не входит в пак спейси
# После этого нужно нажать сверху на кнопку runtime и выбрать restart runtime
!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.1.0/ru_core_news_sm-3.1.0.tar.gz

     |████████████████████████████████| 15.9 MB 138 kB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 668 kB/s eta 0:00:01
     |████████████████████████████████| 97 kB 1.1 MB/s eta 0:00:01
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.0
    Uninstalling spacy-3.2.0:
      Successfully uninstalled spacy-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ru-core-news-md 3.2.0 requires spacy<3.3.0,>=3.2.0, but you have spacy 3.1.6 which is incompatible.


In [4]:
import spacy

In [5]:
import csv

In [6]:
nlp = spacy.load("ru_core_news_sm")

In [7]:
nlp.max_length = 20000000

In [8]:
doc_spacy = nlp(book)

In [9]:
header = ['spacy', 'start']
rows = []

In [10]:
maybe_names = []

In [11]:
for ent in doc_spacy.ents:
    if ent.label_ == 'LOC':
        one_row = []
        one_row = [ent.lemma_, ent.start_char]
        rows.append(one_row)
    else:
        if ent.label_ == 'PER':
        maybe_names.append(ent.lemma_)

In [12]:
rows

[['припепешинский переулок', 21],
 ['мюр-мерилиза', 463],
 ['арбат', 1000],
 ['митю', 1088],
 ['ваганьково', 1509],
 ['табачихинский переулок', 2294],
 ['табачихинского переулок', 2529],
 ['табачихинский переулок', 2766]]

In [13]:
maybe_names

['канашке лиза',
 'митя коробкин',
 'уж митя',
 'грибиков',
 'арбат',
 'грибиков',
 'грибиков',
 'грибиков',
 'митя',
 'грибиков',
 'да мамаша',
 'да надежда иванович',
 'грибиков',
 'коробкин',
 'коробкин',
 'грибиков']

In [28]:
with open('results_spacy.csv','w') as file_spacy:
  writer = csv.writer(file_spacy)
  writer.writerow(header)
  writer.writerows(rows)

# 4. Работаем с Natasha

In [14]:
!pip install natasha

In [15]:
import natasha

In [16]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [17]:
doc_natasha = Doc(book)

In [18]:
doc_natasha.segment(segmenter)

In [19]:
doc_natasha.tag_morph(morph_tagger)
doc_natasha.parse_syntax(syntax_parser)

In [20]:
doc_natasha.tag_ner(ner_tagger)

In [21]:
locations = []

In [22]:
supposedly_names = []

In [23]:
for span in doc_natasha.spans:
  if span.type == 'LOC':
    span.normalize(morph_vocab)
    locations.append(span)
  else:
    if span.type == "PER":
      span.normalize(morph_vocab)
      supposedly_names.append(span.normal)

In [24]:
supposedly_names

['Канашке Лиза От Мюр-Мерилиз',
 'Митя Коробкин',
 'Грибиков',
 'Митя',
 'Грибиков',
 'Грибиков',
 'Митя',
 'Грибиков',
 'Надежда Ивановна',
 'Грибиков',
 'Коробкин',
 'Коробкин',
 'Грибиков']

In [25]:
header_natasha = ['natasha', 'start']
rows_natasha =[]

In [26]:
for span in locations:
  one_row = []
  one_row = [span.normal, span.start]
  rows_natasha.append(one_row)

In [27]:
with open('results_natasha.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(header_natasha)
    writer.writerows(rows_natasha)

# 5. Объединяем две таблички (outer) для сравнения


In [37]:
import pandas as pd

In [38]:
natasha_table = pd.read_csv("results_natasha.csv")

In [39]:
natasha_table

,natasha,start
0,Припепешинский переулок,21
1,Припепешинским,182
2,Арбат,842
3,Арбат,1000
4,Митю,1088
5,Ваганьково,1509
6,Табачихинский переулок,2294
7,Табачихинский переулок,2529
8,Табачихинский переулок,2766
9,Табачихинский переулок,3050


In [40]:
spacy_table = pd.read_csv("results_spacy.csv")

In [41]:
spacy_table

,spacy,start
0,припепешинский переулок,21
1,мюр-мерилиза,463
2,арбат,1000
3,митю,1088
4,ваганьково,1509
5,табачихинский переулок,2294
6,табачихинского переулок,2529
7,табачихинский переулок,2766


In [42]:
merged_tables_outer = pd.merge(natasha_table, spacy_table, on= "start", how = "outer")

In [43]:
merged_tables_outer

,natasha,start,spacy
0,Припепешинский переулок,21,припепешинский переулок
1,Припепешинским,182,NaN
2,Арбат,842,NaN
3,Арбат,1000,арбат
4,Митю,1088,митю
5,Ваганьково,1509,ваганьково
6,Табачихинский переулок,2294,табачихинский переулок
7,Табачихинский переулок,2529,табачихинского переулок
8,Табачихинский переулок,2766,табачихинский переулок
9,Табачихинский переулок,3050,NaN


In [44]:
column_names = ["start", "natasha", "spacy"]

In [45]:
outer_table = merged_tables_outer.reindex(columns = column_names)

In [46]:
outer_table.to_csv("outer_table.csv", index=False)

# 6. Объединяем таблички (inner)

In [47]:
merged_tables_inner = pd.merge(natasha_table, spacy_table, on= "start", how = "inner")

In [48]:
merged_tables_inner

,natasha,start,spacy
0,Припепешинский переулок,21,припепешинский переулок
1,Арбат,1000,арбат
2,Митю,1088,митю
3,Ваганьково,1509,ваганьково
4,Табачихинский переулок,2294,табачихинский переулок
5,Табачихинский переулок,2529,табачихинского переулок
6,Табачихинский переулок,2766,табачихинский переулок


In [49]:
inner_table = merged_tables_inner.reindex(columns = column_names)

In [50]:
inner_table.to_csv("inner_table.csv", index=False)

# 7. Повторная лемматизация локаций, найденных Spacy и Наташей

In [51]:
inner_table

,start,natasha,spacy
0,21,Припепешинский переулок,припепешинский переулок
1,1000,Арбат,арбат
2,1088,Митю,митю
3,1509,Ваганьково,ваганьково
4,2294,Табачихинский переулок,табачихинский переулок
5,2529,Табачихинский переулок,табачихинского переулок
6,2766,Табачихинский переулок,табачихинский переулок


In [52]:
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install DAWG-Python

In [53]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [54]:
twice_normalized = []

In [55]:
for i in inner_table['spacy']:
  p = morph.parse(i)[0]
  twice_normalized.append(p.normal_form)

In [56]:
twice_normalized

['припепешинский переулка',
 'арбат',
 'митя',
 'ваганьковый',
 'табачихинский переулка',
 'табачихинского переулка',
 'табачихинский переулка']

In [58]:
spacy_twice = inner_table[['start','natasha', 'spacy']]
spacy_twice['spacy'] = twice_normalized
spacy_twice

,start,natasha,spacy
0,21,Припепешинский переулок,припепешинский переулка
1,1000,Арбат,арбат
2,1088,Митю,митя
3,1509,Ваганьково,ваганьковый
4,2294,Табачихинский переулок,табачихинский переулка
5,2529,Табачихинский переулок,табачихинского переулка
6,2766,Табачихинский переулок,табачихинский переулка


# 6. Подгружаем черный список 


## Черный список географических объектов

In [59]:
with open('new_black_list.csv', 'r') as read_obj:
  b_list = csv.reader(read_obj)
  list_of_csv = list(b_list)
  print(list_of_csv)

[['Абхазия'], ['Австралия'], ['Австрия'], ['Азербайджан'], ['Албания'], ['Алжир'], ['Ангола'], ['Англия'], ['Андорра'], ['Антигуа и Барбуда'], ['Аргентина'], ['Армения'], ['Афганистан'], ['Багамские Острова'], ['Багамы'], ['Бангладеш'], ['Барбадос'], ['Бахрейн'], ['Белиз'], ['Белоруссия'], ['Бельгия'], ['Бенин'], ['Болгария'], ['Боливия'], ['Босния и Герцеговина'], ['Ботсвана'], ['Бразилия'], ['Бруней'], ['Буркина-Фасо'], ['Бурунди'], ['Бутан'], ['Вануату'], ['Ватикан'], ['Великобритания'], ['Венгрия'], ['Венесуэла'], ['Восточный Тимор'], ['Вьетнам'], ['Габон'], ['Гаити'], ['Гайана'], ['Гамбия'], ['Гана'], ['Гватемала'], ['Гвинея'], ['Гвинея-Бисау'], ['Германия'], ['Гондурас'], ['Палестина'], ['Гренада'], ['Греция'], ['Грузия'], ['Дания'], ['Джибути'], ['Доминика'], ['Доминиканская Республика'], ['Конго'], ['Египет'], ['Замбия'], ['Зимбабве'], ['Израиль'], ['Индия'], ['Индонезия'], ['Иордания'], ['Ирак'], ['Иран'], ['Ирландия'], ['Исландия'], ['Испания'], ['Италия'], ['Йемен'], ['Кабо-

In [72]:
black_list = []
for i in list_of_csv:
  loc = ''.join(i).lower()
  black_list.append(loc)

In [73]:
len(black_list)

16287

## Черный список имен 

In [74]:
with open('names_rus.csv', 'r') as f:
  n_list = csv.reader(f)
  list_of_names = list(n_list)
  print(list_of_names)

[['Name'], ['Аалия'], ['Аанжелла'], ['Аба'], ['Абав'], ['Абам'], ['Абаханум'], ['Абаш'], ['Аббаза'], ['Абгюль'], ['Абдужалия'], ['Абдулния'], ['Абдулпетта'], ['Абдулька'], ['Абдупатта'], ['Абдурасуль'], ['Абдурефи'], ['Абдурома'], ['Абдюль'], ['Абелина'], ['Абель'], ['Абибат'], ['Абигейль'], ['Абигель'], ['Абика'], ['Абилфетта'], ['Аблялиль'], ['Абрагина'], ['Абрамия'], ['Абраша'], ['Абрииа'], ['Абуватфа'], ['Абузарь'], ['Абульфаза'], ['Абуся'], ['Абута'], ['Абутаима'], ['Авагея'], ['Аванда'], ['Аванна'], ['Авашарфа'], ['Авашерфа'], ['Аващарфа'], ['Авгалифена'], ['Авгела'], ['Авгения'], ['Авгиння'], ['Авгусита'], ['Августа'], ['Августина'], ['Августиния'], ['Авдия'], ['Авдокия'], ['Авдоля'], ['Авдотия'], ['Авдотья'], ['Авдоха'], ['Авелина'], ['Авелия'], ['Авеля'], ['Авенгилина'], ['Авенила'], ['Авения'], ['Авентина'], ['Авера'], ['Авереса'], ['Аверида'], ['Аверия'], ['Авета'], ['Аветлана'], ['Аветра'], ['Авза'], ['Авзаль'], ['Авиа'], ['Авиада'], ['Авиана'], ['Авиата'], ['Авиатта'], ['А

In [79]:
names_black_list =[]

In [80]:
for i in list_of_names:
    loct = ''.join(i).lower()
    names_black_list.append(loct)

In [81]:
len(names_black_list)

51530

## Добавляем извлеченные имена

In [82]:
for i in extracted_names:
    if i not in names_black_list:
        names_black_list.append(i)

In [83]:
len(names_black_list)

51530

## Библиотека russiannames

In [84]:
import pymongo

In [85]:
import click

In [86]:
!pip install russiannames

In [91]:
from russiannames.parser import NamesParser
parser = NamesParser()

In [95]:
for i in book.split():
    a = parser.parse(i)['parsed']
    if a == True:
        if i not in names_black_list:
            names_black_list.append(i)

In [96]:
len(names_black_list)

51686

## Объединяем черные списки

In [97]:
len(black_list)

16287

In [98]:
black_list.extend(names_black_list)

In [99]:
len(black_list)

67973

# 7. Отсеиваем результаты inner join по черному списку

### 7.1. Начинаем со Spacy

In [106]:
header_clean_spacy = ['spacy', 'start']
rows_clean_spacy = []

for ent in doc_spacy.ents:
    if ent.label_ == 'LOC':
        p = morph.parse(ent.lemma_)[0]
        if p.normal_form not in black_list:
            one_row = []
            one_row = [ent.lemma_, ent.start_char]
            rows_clean_spacy.append(one_row)

In [107]:
with open('results_spacy_clean.csv','w') as file_spacy_clean:
  writer = csv.writer(file_spacy_clean)
  writer.writerow(header_clean_spacy)
  writer.writerows(rows_clean_spacy)

### 7.2. Проделываем то же самое с кодом Natasha

In [109]:
header_clean_natasha = ['natasha', 'start']
rows_clean_natasha = []

for span in locations:
    p = morph.parse(span.normal)[0]
    if p.normal_form.lower() not in black_list:
        one_row = []
        one_row = [span.normal, span.start]
        rows_clean_natasha.append(one_row)

In [110]:
rows_clean_natasha

[['Припепешинский переулок', 21],
 ['Припепешинским', 182],
 ['Арбат', 842],
 ['Арбат', 1000],
 ['Ваганьково', 1509],
 ['Табачихинский переулок', 2294],
 ['Табачихинский переулок', 2529],
 ['Табачихинский переулок', 2766],
 ['Табачихинский переулок', 3050]]

In [111]:
with open('results_natasha_clean.csv', 'w') as file:
  writer = csv.writer(file)
  writer.writerow(header_clean_natasha)
  writer.writerows(rows_clean_natasha)

### 7.3 Объединяем две таблички 

In [112]:
natasha_table_clean = pd.read_csv("results_natasha_clean.csv")

In [113]:
spacy_table_clean = pd.read_csv("results_spacy_clean.csv")

In [114]:
clean_tables_inner = pd.merge(natasha_table_clean, spacy_table_clean, on= "start", how = "inner")

In [115]:
clean_tables_inner

,natasha,start,spacy
0,Припепешинский переулок,21,припепешинский переулок
1,Арбат,1000,арбат
2,Ваганьково,1509,ваганьково
3,Табачихинский переулок,2294,табачихинский переулок
4,Табачихинский переулок,2529,табачихинского переулок
5,Табачихинский переулок,2766,табачихинский переулок


In [116]:
clean_tables_inner.to_csv("clean_results.csv", index=False)

### 7.4. Сравниваем результы двух таблиц *inner_table.txt* и *clean_results.txt*

In [117]:
merge_new = pd.merge(inner_table, clean_tables_inner, on = 'start', how = 'outer')

In [118]:
merge_new

,start,natasha_x,spacy_x,natasha_y,spacy_y
0,21,Припепешинский переулок,припепешинский переулок,Припепешинский переулок,припепешинский переулок
1,1000,Арбат,арбат,Арбат,арбат
2,1088,Митю,митю,NaN,NaN
3,1509,Ваганьково,ваганьково,Ваганьково,ваганьково
4,2294,Табачихинский переулок,табачихинский переулок,Табачихинский переулок,табачихинский переулок
5,2529,Табачихинский переулок,табачихинского переулок,Табачихинский переулок,табачихинского переулок
6,2766,Табачихинский переулок,табачихинский переулок,Табачихинский переулок,табачихинский переулок


In [119]:
tables = merge_new.rename(columns={'natasha_x': 'natasha', 'spacy_x': 'spacy', 'natasha_y': 'natasha_clean', 'spacy_y': 'spacy_clean'})

In [120]:
tables

,start,natasha,spacy,natasha_clean,spacy_clean
0,21,Припепешинский переулок,припепешинский переулок,Припепешинский переулок,припепешинский переулок
1,1000,Арбат,арбат,Арбат,арбат
2,1088,Митю,митю,NaN,NaN
3,1509,Ваганьково,ваганьково,Ваганьково,ваганьково
4,2294,Табачихинский переулок,табачихинский переулок,Табачихинский переулок,табачихинский переулок
5,2529,Табачихинский переулок,табачихинского переулок,Табачихинский переулок,табачихинского переулок
6,2766,Табачихинский переулок,табачихинский переулок,Табачихинский переулок,табачихинский переулок


In [121]:
tables.to_csv("both_results.csv", index=False)